In [12]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext, DataFrameWriter
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[8]").getOrCreate()
sqlc = SQLContext(spark)

/home/gorgeous/Documents/.venv/lib/python3.8/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
data_flights  = spark.read.option("header", "true") \
      .option("inferSchema", "true") \
      .csv("/user/gorgeous/Flights/rawzone/tech_year=2023/tech_month=2023-6/tech_day=2023-6-6/*.csv")

data_airports = spark.read.option("header", "true") \
      .option("inferSchema", "true") \
      .csv("/user/gorgeous/Airports/rawzone/tech_year=2023/tech_month=2023-6/tech_day=2023-6-6/*.csv")

data_airlines = spark.read.option("header", "true") \
      .option("inferSchema", "true") \
      .csv("/user/gorgeous/Airlines/rawzone/tech_year=2023/tech_month=2023-6/tech_day=2023-6-6/*.csv")


In [3]:
data_flights_batch = data_flights.sample(0.02)

data_flights_batch.createOrReplaceTempView("data_flights_batch")
airports_iata = sqlc.sql("""select origin_airport_iata from data_flights_batch UNION 
                            select destination_airport_iata from data_flights_batch""")


airports_iata = airports_iata.distinct()
airports_iata.createOrReplaceTempView("airports_iata")

In [4]:
data_airports.createOrReplaceTempView("data_airports")
data_airports_batch = sqlc.sql("""select * from data_airports where iata IN (select * from airports_iata) """)


In [5]:
airlines_iata = sqlc.sql("""select airline_iata from data_flights_batch""")

airlines_iata = airlines_iata.distinct()
airlines_iata.createOrReplaceTempView("airlines_iata")

In [6]:
data_airlines.createOrReplaceTempView("data_airlines")
data_airlines_batch = sqlc.sql("""select * from data_airlines where Code IN (select * from airlines_iata) """)

In [11]:
data_flights_batch.coalesce(1).write.option("header", True).csv("/flights_batch")
data_airports_batch.coalesce(1).write.option("header", True).csv("/airports_batch")
data_airlines_batch.coalesce(1).write.option("header", True).csv("/airlines_batch")